<a href="https://colab.research.google.com/github/vikilaboy/FlexiJsonEditor/blob/master/RealTimeFeedbackSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Packages

In [ ]:
!pip install -q openai==1.43.0 pandas==2.2.2 pydantic==2.8.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is incompatible.


## OpenAI API Key

In [ ]:
import getpass
from openai import OpenAI

OPENAI_API_KEY = getpass.getpass('OpenAI API Key: ')

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

OpenAI API Key: ··········


# Response Formats

Structured Outputs documentation:
- https://platform.openai.com/docs/guides/structured-outputs/examples
- https://openai.com/index/introducing-structured-outputs-in-the-api/

## Issues

In [ ]:
from pydantic import BaseModel, Field

class Issue(BaseModel):
    explanation: str = Field(description="A brief explanation of the issue.")
    title: str = Field(description="A short title for the issue.")
    example_ids: list[str] = Field(description="A list of feedback IDs which discuss the issue, such as F1, F2, etc.")
    mentioned_employees: list[str] = Field(description="A list of employees mentioned in examples of the issue.")
    potential_solutions: list[str] = Field(description="A list of potential solutions to the issue.")

class IssueResponse(BaseModel):
    issues: list[Issue] = Field(description="A list of issues. Start with the most significant and/or widespread issues.")
    summary: str = Field(description="A short high-level summary of the most significant and/or widespread issues.")


## Success Stories

In [ ]:
from pydantic import BaseModel, Field

class SuccessStory(BaseModel):
    explanation: str = Field(description="A brief explanation of the success story.")
    title: str = Field(description="A short title for the success story.")
    example_ids: list[str] = Field(description="A list of feedback IDs which discuss the positive feedback, such as F1, F2, etc.")
    mentioned_employees: list[str] = Field(description="A list of employees mentioned in examples of the feedback.")
    potential_insights: list[str] = Field(description="A list of potential insights to improve the experience further, or capitalize on the success.")

class SuccessResponse(BaseModel):
    success_stories: list[SuccessStory] = Field(description="A list of success stories. Start with the most positive and/or widespread success stories.")
    summary: str = Field(description="A short high-level summary of the most positive and/or widespread success stories.")


# Upload Excel

In [ ]:
from google.colab import files

files = files.upload()

Saving phillips_toyota_feedbacks_from_07_01_2024_to_08_01_2024.csv to phillips_toyota_feedbacks_from_07_01_2024_to_08_01_2024.csv


In [ ]:
import pandas as pd

df = pd.read_csv('phillips_toyota_feedbacks_from_07_01_2024_to_08_01_2024.csv')
# df = pd.read_excel('golds_gym_the_pike_feedbacks_from_07_01_2024_to_07_31_2024.xlsx')
print(len(df))
df.head()

163


,timest,description
0,2024-07-01 13:48:40,Good service! But I did not receive my multi ...
1,2024-07-01 16:10:47,"Trace does an superb job and as always, is inf..."
2,2024-07-01 19:08:13,Phillips Toyota just completed my 50K service ...
3,2024-07-01 20:53:55,I’m curious what the $22 listed as shop suppli...
4,2024-07-01 22:00:19,The actual car service was great. The service ...


In [ ]:
print(df['timest'].iloc[0])
print(df['timest'].iloc[-1])


2024-07-01 13:48:40
2024-07-31 22:57:12


In [ ]:
# feedback = {
#     'F1': 'Good service! But I did not receive my multi point inspection sheet. Those are really valuable. Colonel Knox',
#     'F2': 'Trace does an superb job and as always, is informative, efficient and professional. He is an excellent asset to the Service Department.',
#     'F3': 'I’m curious what the $22 listed as shop supplies has to do with me buying a battery and carrying it out? It gives me an uneasy feeling if I was to come back for some mechanical work, what extra fees would be added.',
# }

In [ ]:
feedback = {}
for idx, f_text in enumerate(df['description']):
    feedback[f'F{idx+1}'] = f_text

In [ ]:
len(feedback)

163

# Issues

In [ ]:
location_type = 'dealership'

system_prompt = f"""
You are a helpful customer service assistant.
You will be tasked with identifying, organizing, and summarizing the shared issues in the following feedback.
The feedback is from a {location_type}.
Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
""".strip()

system_prompt = ' '.join(system_prompt.split('\n'))
print(system_prompt)


user_prompt = "Please identify the issues in the following feedback:"

feedback_str_lines = []
for f_id, f_text in feedback.items():
    feedback_str_lines.append(f"{f_id}: {f_text}")
feedback_str = '\n\n'.join(feedback_str_lines)

feedback_prompt = f"""{user_prompt}
{feedback_str}
"""

print(feedback_prompt)


# gpt-4o-2024-08-06 or gpt-4o-mini
model_name = 'gpt-4o-2024-08-06'


completion = client.beta.chat.completions.parse(
    model=model_name,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": feedback_prompt},
    ],
    response_format=IssueResponse,
    seed=0,
    top_p=0.3,
    temperature=1.0,
)
model_response = completion.choices[0].message
response = model_response.parsed

You are a helpful customer service assistant. You will be tasked with identifying, organizing, and summarizing the shared issues in the following feedback. The feedback is from a dealership. Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
Please identify the issues in the following feedback:
F1: Good service!  But I did not receive my multi point inspection sheet.   Those are really valuable.   Colonel Knox

F2: Trace does an superb job and as always, is informative, efficient and professional. He is an excellent asset to the Service Department.

F3: Phillips Toyota just completed my 50K service and a couple other items recommended by Toyota based on mileage.  The service was completed in a timely manner and I was out the door in less than two hours.  Would definitely recommend Phillips to anyone seeking service on a Toyota vehicle.

F4: I’m curious what the $22 listed as shop supplies has to do with me buying a battery and c

In [ ]:
# took 30 seconds to generate from 163 examples of feedback. Cost ~$0.02 per report

In [ ]:
print(response.summary)
for issue in response.issues:
    print(f"  {issue.title} ({len(issue.example_ids):,} messages)")
    print(f"    {issue.explanation}")
    if issue.mentioned_employees:
        print(f"    Mentioned Employees: {', '.join(issue.mentioned_employees)}")
    if issue.potential_solutions:
        print(f"    Potential Solutions:")
        for solution in issue.potential_solutions:
            print(f"      {solution}")
    print(f"    Examples:")
    for example_id in issue.example_ids:
        print(f"      {example_id}: {feedback[example_id]}")
    print()

The most significant issues identified from the feedback include long wait times for service appointments, unexpected charges without prior communication, pressure to purchase unnecessary services, incomplete or incorrect service leading to additional visits, and poor communication from staff. Addressing these issues could improve customer satisfaction and service efficiency.
  Long Wait Times (37 messages)
    Customers frequently experience long wait times for their scheduled appointments, often extending beyond the expected duration for the services being performed.
    Potential Solutions:
      Improve scheduling to prevent overbooking.
      Increase staffing during peak hours to reduce wait times.
      Implement a more efficient workflow to expedite service times.
    Examples:
      F8: My appointment was at 4. I was on time. For the services I needed did NOT 2hrs. to complete! Very disappointed.
      F9: Very long time service I am not happy .then he can change my car oil be

# Success Stories

In [ ]:
location_type = 'dealership'

system_prompt = f"""
You are a helpful customer service assistant.
You will be tasked with identifying, organizing, and summarizing the shared success stories in the following feedback.
The feedback is from a {location_type}.
Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
""".strip()

system_prompt = ' '.join(system_prompt.split('\n'))
print(system_prompt)


user_prompt = "Please identify the success stories in the following feedback:"

feedback_str_lines = []
for f_id, f_text in feedback.items():
    feedback_str_lines.append(f"{f_id}: {f_text}")
feedback_str = '\n\n'.join(feedback_str_lines)

feedback_prompt = f"""{user_prompt}
{feedback_str}
"""

print(feedback_prompt)


# gpt-4o-2024-08-06 or gpt-4o-mini
model_name = 'gpt-4o-2024-08-06'


completion = client.beta.chat.completions.parse(
    model=model_name,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": feedback_prompt},
    ],
    response_format=SuccessResponse,
    seed=0,
    top_p=0.3,
    temperature=1.0,
)
model_response = completion.choices[0].message
response = model_response.parsed

You are a helpful customer service assistant. You will be tasked with identifying, organizing, and summarizing the shared success stories in the following feedback. The feedback is from a dealership. Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
Please identify the success stories in the following feedback:
F1: Good service!  But I did not receive my multi point inspection sheet.   Those are really valuable.   Colonel Knox

F2: Trace does an superb job and as always, is informative, efficient and professional. He is an excellent asset to the Service Department.

F3: Phillips Toyota just completed my 50K service and a couple other items recommended by Toyota based on mileage.  The service was completed in a timely manner and I was out the door in less than two hours.  Would definitely recommend Phillips to anyone seeking service on a Toyota vehicle.

F4: I’m curious what the $22 listed as shop supplies has to do with me buyi

In [ ]:
# 21 seconds

In [ ]:
print(response.summary)
for story in response.success_stories:
    print(f"  {story.title} ({len(story.example_ids):,} messages)")
    print(f"    {story.explanation}")
    if story.mentioned_employees:
        print(f"    Mentioned Employees: {', '.join(story.mentioned_employees)}")
    if story.potential_insights:
        print(f"    Potential Insights:")
        for solution in story.potential_insights:
            print(f"      {solution}")
    print(f"    Examples:")
    for example_id in story.example_ids:
        print(f"      {example_id}: {feedback[example_id]}")
    print()

The dealership has received positive feedback for its outstanding service provided by employees like Trace and Rhonda Cook, who are noted for their professionalism and customer service. Additionally, the dealership is praised for its efficient and timely service, as well as the friendly and helpful nature of its staff.
  Outstanding Service by Trace (2 messages)
    Trace is consistently praised for his professionalism, efficiency, and informative nature, making him a valuable asset to the service department.
    Mentioned Employees: Trace
    Potential Insights:
      Consider recognizing Trace's contributions formally to encourage similar behavior in other employees.
      Use Trace's approach as a model for training other service staff.
    Examples:
      F2: Trace does an superb job and as always, is informative, efficient and professional. He is an excellent asset to the Service Department.
      F99: Excellent , caring  personnel, everyone we were in contact with just great, esp

# Gym

In [ ]:
location_type = 'gym'

system_prompt = f"""
You are a helpful customer service assistant.
You will be tasked with identifying, organizing, and summarizing the shared issues in the following feedback.
The feedback is from a {location_type}.
Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
""".strip()

system_prompt = ' '.join(system_prompt.split('\n'))
print(system_prompt)


user_prompt = "Please identify the issues in the following feedback:"

feedback_str_lines = []
for f_id, f_text in feedback.items():
    feedback_str_lines.append(f"{f_id}: {f_text}")
feedback_str = '\n\n'.join(feedback_str_lines)

feedback_prompt = f"""{user_prompt}
{feedback_str}
"""

print(feedback_prompt)


# gpt-4o-2024-08-06 or gpt-4o-mini
model_name = 'gpt-4o-2024-08-06'


completion = client.beta.chat.completions.parse(
    model=model_name,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": feedback_prompt},
    ],
    response_format=IssueResponse,
    seed=0,
    top_p=0.3,
    temperature=1.0,
)
model_response = completion.choices[0].message
response = model_response.parsed

You are a helpful customer service assistant. You will be tasked with identifying, organizing, and summarizing the shared issues in the following feedback. The feedback is from a gym. Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
Please identify the issues in the following feedback:
F1: Hi thanks for asking. The gummy looks beautiful and the staff are great. I noticed two machines were broken, one of them I’m has been for over two weeks and it is on of only two you have In the gym.    I also noticed that there aren’t enough disinfectant spray bottles and the hand towels can go empty for a while before someone replenishes them.   Other than that 👍👍👍

F2: Great gym with great staff! The reason I’m giving 4-stars is because there was a line for some of the machines. Maybe raise the memberships so there’s less crowds.

F3: 3 different sprays were out in stations and people stopped cleaning their equipment

F4: Even at 10pm the 

In [ ]:
print(response.summary)
for issue in response.issues:
    print(f"  {issue.title} ({len(issue.example_ids):,} messages)")
    print(f"    {issue.explanation}")
    if issue.mentioned_employees:
        print(f"    Mentioned Employees: {', '.join(issue.mentioned_employees)}")
    if issue.potential_solutions:
        print(f"    Potential Solutions:")
        for solution in issue.potential_solutions:
            print(f"      {solution}")
    print(f"    Examples:")
    for example_id in issue.example_ids:
        print(f"      {example_id}: {feedback[example_id]}")
    print()

The most significant issues at the gym include overcrowding, broken equipment, and cleanliness concerns. Members also report problems with loud music, temperature control, inappropriate behavior, parking limitations, and customer service. Addressing these issues could improve the overall member experience.
  Overcrowding (17 messages)
    Many members report that the gym is overcrowded, making it difficult to access equipment and complete workouts efficiently.
    Potential Solutions:
      Limit the number of new memberships sold.
      Implement a reservation system for peak hours.
      Increase the number of popular machines.
    Examples:
      F2: Great gym with great staff! The reason I’m giving 4-stars is because there was a line for some of the machines. Maybe raise the memberships so there’s less crowds.
      F4: Even at 10pm the gym was way too crowded to even try to get a decent workout.   You guys sold way too many heavily discounted memberships and the quality/maturity o

In [ ]:
import json

with open('gym_issues.json', 'w') as f:
    json.dump(response.dict(), f, indent=2)

In [ ]:
location_type = 'gym'

system_prompt = f"""
You are a helpful customer service assistant.
You will be tasked with identifying, organizing, and summarizing the shared success stories in the following feedback.
The feedback is from a {location_type}.
Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
""".strip()

system_prompt = ' '.join(system_prompt.split('\n'))
print(system_prompt)


user_prompt = "Please identify the success stories in the following feedback:"

feedback_str_lines = []
for f_id, f_text in feedback.items():
    feedback_str_lines.append(f"{f_id}: {f_text}")
feedback_str = '\n\n'.join(feedback_str_lines)

feedback_prompt = f"""{user_prompt}
{feedback_str}
"""

print(feedback_prompt)


# gpt-4o-2024-08-06 or gpt-4o-mini
model_name = 'gpt-4o-2024-08-06'


completion = client.beta.chat.completions.parse(
    model=model_name,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": feedback_prompt},
    ],
    response_format=SuccessResponse,
    seed=0,
    top_p=0.3,
    temperature=1.0,
)
model_response = completion.choices[0].message
response = model_response.parsed

You are a helpful customer service assistant. You will be tasked with identifying, organizing, and summarizing the shared success stories in the following feedback. The feedback is from a gym. Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
Please identify the success stories in the following feedback:
F1: Hi thanks for asking. The gummy looks beautiful and the staff are great. I noticed two machines were broken, one of them I’m has been for over two weeks and it is on of only two you have In the gym.    I also noticed that there aren’t enough disinfectant spray bottles and the hand towels can go empty for a while before someone replenishes them.   Other than that 👍👍👍

F2: Great gym with great staff! The reason I’m giving 4-stars is because there was a line for some of the machines. Maybe raise the memberships so there’s less crowds.

F3: 3 different sprays were out in stations and people stopped cleaning their equipment

F4:

In [ ]:
print(response.summary)
for story in response.success_stories:
    print(f"  {story.title} ({len(story.example_ids):,} messages)")
    print(f"    {story.explanation}")
    if story.mentioned_employees:
        print(f"    Mentioned Employees: {', '.join(story.mentioned_employees)}")
    if story.potential_insights:
        print(f"    Potential Insights:")
        for solution in story.potential_insights:
            print(f"      {solution}")
    print(f"    Examples:")
    for example_id in story.example_ids:
        print(f"      {example_id}: {feedback[example_id]}")
    print()

Members of the gym frequently praise the friendly and welcoming staff, the cleanliness and maintenance of the facilities, and the variety and availability of equipment. These aspects significantly enhance their workout experience and satisfaction with the gym.
  Friendly and Welcoming Staff (12 messages)
    Many members appreciate the friendly and welcoming staff at the gym, which contributes positively to their overall experience.
    Mentioned Employees: Chris
    Potential Insights:
      Continue to train staff on customer service to maintain and enhance the welcoming atmosphere.
      Recognize and reward staff members who consistently receive positive feedback from members.
    Examples:
      F1: Hi thanks for asking. The gummy looks beautiful and the staff are great. I noticed two machines were broken, one of them I’m has been for over two weeks and it is on of only two you have In the gym.    I also noticed that there aren’t enough disinfectant spray bottles and the hand towe

In [ ]:
import json

with open('gym_success.json', 'w') as f:
    json.dump(response.dict(), f, indent=2)

# Live Demo

In [ ]:
location_type = 'dealership'

system_prompt = f"""
You are a helpful customer service assistant.
You will be tasked with identifying, organizing, and summarizing the shared issues in the following feedback.
The feedback is from a {location_type}.
Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
""".strip()

system_prompt = ' '.join(system_prompt.split('\n'))
print(system_prompt)


user_prompt = "Please identify the issues in the following feedback:"

feedback_str_lines = []
for f_id, f_text in feedback.items():
    feedback_str_lines.append(f"{f_id}: {f_text}")
feedback_str = '\n\n'.join(feedback_str_lines)

feedback_prompt = f"""{user_prompt}
{feedback_str}
"""

print(feedback_prompt)


# gpt-4o-2024-08-06 or gpt-4o-mini
model_name = 'gpt-4o-2024-08-06'


completion = client.beta.chat.completions.parse(
    model=model_name,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": feedback_prompt},
    ],
    response_format=IssueResponse,
    seed=0,
    top_p=0.3,
    temperature=1.0,
)
model_response = completion.choices[0].message
response = model_response.parsed

You are a helpful customer service assistant. You will be tasked with identifying, organizing, and summarizing the shared issues in the following feedback. The feedback is from a dealership. Be sure to use the correct feedback IDs (F1, F2, etc.) when referencing the feedback in supporting examples.
Please identify the issues in the following feedback:
F1: Good service!  But I did not receive my multi point inspection sheet.   Those are really valuable.   Colonel Knox

F2: Trace does an superb job and as always, is informative, efficient and professional. He is an excellent asset to the Service Department.

F3: Phillips Toyota just completed my 50K service and a couple other items recommended by Toyota based on mileage.  The service was completed in a timely manner and I was out the door in less than two hours.  Would definitely recommend Phillips to anyone seeking service on a Toyota vehicle.

F4: I’m curious what the $22 listed as shop supplies has to do with me buying a battery and c

In [ ]:
schema = IssueResponse.model_json_schema()
schema

{'$defs': {'Issue': {'properties': {'explanation': {'description': 'A brief explanation of the issue.',
     'title': 'Explanation',
     'type': 'string'},
    'title': {'description': 'A short title for the issue.',
     'title': 'Title',
     'type': 'string'},
    'example_ids': {'description': 'A list of feedback IDs which discuss the issue, such as F1, F2, etc.',
     'items': {'type': 'string'},
     'title': 'Example Ids',
     'type': 'array'},
    'mentioned_employees': {'description': 'A list of employees mentioned in examples of the issue.',
     'items': {'type': 'string'},
     'title': 'Mentioned Employees',
     'type': 'array'},
    'potential_solutions': {'description': 'A list of potential solutions to the issue.',
     'items': {'type': 'string'},
     'title': 'Potential Solutions',
     'type': 'array'}},
   'required': ['explanation',
    'title',
    'example_ids',
    'mentioned_employees',
    'potential_solutions'],
   'title': 'Issue',
   'type': 'object'}},

In [ ]:
import json
print(json.dumps(response.dict(), indent=2))

{
  "issues": [
    {
      "explanation": "Many customers reported that their service appointments took significantly longer than expected, with some waiting several hours for simple services like oil changes or tire rotations.",
      "title": "Long Wait Times for Service",
      "example_ids": [
        "F8",
        "F9",
        "F18",
        "F21",
        "F22",
        "F23",
        "F27",
        "F28",
        "F31",
        "F32",
        "F34",
        "F36",
        "F37",
        "F38",
        "F41",
        "F44",
        "F55",
        "F61",
        "F63",
        "F64",
        "F72",
        "F73",
        "F74",
        "F76",
        "F102",
        "F103",
        "F104",
        "F110",
        "F127",
        "F132",
        "F137",
        "F139",
        "F152",
        "F153"
      ],
      "mentioned_employees": [],
      "potential_solutions": [
        "Improve scheduling to prevent overbooking.",
        "Increase staffing during peak times.",
        